# FAC: Field-aligned currents

The FAC toolbox provides the ability to calculate FACs on-demand, given magnetic measurements and model predictions.

Currently, the single-satellite algorithm is implemented and provides very similar output as the operational product [`SW_FACxTMS_2F`](https://swarmhandbook.earth.esa.int/catalogue/SW_FACxTMS_2F). The results are not identical because of differences in the processing chain (for example, using the POMME model instead of the CHAOS model to supply the background magnetic field, or "mean field")

For a description of the method, see:  
Ritter, P., Lühr, H. & Rauberg, J. Determining field-aligned currents with the Swarm constellation mission. Earth Planet Sp 65, 1285–1294 (2013). https://doi.org/10.5047/eps.2013.09.006

For more sophisticated FAC estimates, see https://github.com/ablagau/SwarmFACE

In [ ]:
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

from swarmpal.io import create_paldata, PalDataItem
from swarmpal.toolboxes import fac

## Fetching data

In [ ]:
data_params = dict(
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["B_NEC"],
    models=["CHAOS"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T03:00:00",
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)

data = create_paldata(PalDataItem.from_vires(**data_params))
print(data)

In [ ]:
data.swarmpal.pal_meta

## Applying a process and viewing the results

In [ ]:
process = fac.processes.FAC_singlesat(
    config={
        "dataset": "SW_OPER_MAGA_LR_1B",
        "model_varname": "B_NEC_CHAOS",
        "measurement_varname": "B_NEC",
    },
)
data = data.swarmpal.apply(process)
print(data)

In [ ]:
data.swarmpal.pal_meta

In [ ]:
print(data["SW_OPER_MAGA_LR_1B"]["PAL:FAC_output"])

In [ ]:
data.swarmpal_fac.quicklook();

## Retrying with data subselection

This time we will fetch data with a filter applied to the request from VirES

See [viresclient.SwarmRequest.add_filter](https://viresclient.readthedocs.io/en/latest/api.html#viresclient.SwarmRequest.add_filter) for how these behave. `swarmpal.io.PalDataItem.from_vires` accepts a list of such filters.

NB. there is currently a bug requiring that each filter string is enclosed in parentheses.

In [ ]:
data_params = dict(
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["B_NEC"],
    models=["CHAOS"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T03:00:00",
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
    filters=[
        "((QDLat > 50) OR (QDLat < -50))",  # the algorithm is only valid at high latitude
        "(Flags_B <= 9)",  # Exclude particularly bad data
    ],
)

data = create_paldata(PalDataItem.from_vires(**data_params))
data = data.swarmpal.apply(process)
data.swarmpal_fac.quicklook();

## Comparing with FAC product

In [ ]:
# Fetch the input to the FAC process...
data_params_fac_input = dict(
    collection="SW_OPER_MAGA_LR_1B",
    measurements=["B_NEC"],
    models=["CHAOS"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T03:00:00",
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)
# ... and the FAC product itself
data_params_fac_product = dict(
    collection="SW_OPER_FACATMS_2F",
    measurements=["IRC", "FAC"],
    start_time="2016-01-01T00:00:00",
    end_time="2016-01-01T03:00:00",
    server_url="https://vires.services/ows",
    options=dict(asynchronous=False, show_progress=False),
)
data = create_paldata(
    PalDataItem.from_vires(**data_params_fac_input),
    PalDataItem.from_vires(**data_params_fac_product),
)

# Apply the FAC process
process = fac.processes.FAC_singlesat(
    config={
        "dataset": "SW_OPER_MAGA_LR_1B",
        "model_varname": "B_NEC_CHAOS",
        "measurement_varname": "B_NEC",
    },
)
data = data.swarmpal.apply(process)

In [ ]:
# Plot comparing them
fig, axes = plt.subplots(nrows=3, figsize=(15, 10), sharex=True)
data["SW_OPER_MAGA_LR_1B"]["PAL:FAC_output"]["IRC"].plot(ax=axes[0])
data["SW_OPER_FACATMS_2F"]["IRC"].plot(ax=axes[1])
(
    data["SW_OPER_MAGA_LR_1B"]["PAL:FAC_output"]["IRC"]
    - data["SW_OPER_FACATMS_2F"]["IRC"]
).plot(ax=axes[2])
axes[0].set_ylabel(f"SwarmPAL on-demand\n{axes[0].get_ylabel()}")
axes[1].set_ylabel(f"Official product\n{axes[1].get_ylabel()}")
axes[2].set_ylabel("Difference between the above")
for ax in axes:
    ax.grid()
    ax.set_xlabel("")